In [7]:
!pip install -q kaggle
!pip install opendatasets

In [ ]:
import os
import shutil
import random
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.utils import load_img, img_to_array, array_to_img
from google.colab import files
import opendatasets as od

In [5]:
files.upload()

In [4]:
od.download(
    "https://www.kaggle.com/datasets/emmarex/plantdisease")

Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease


100%|██████████| 658M/658M [00:01<00:00, 369MB/s]


In [6]:
original_dir = '/content/plantdisease/plantvillage/PlantVillage'
split_dir = '/content/plantvillage_split'
split_ratio = 0.2
IMG_SIZE = (128, 128)

os.makedirs(split_dir, exist_ok=True)
for split in ['train', 'val']:
    os.makedirs(os.path.join(split_dir, split), exist_ok=True)


for class_name in os.listdir(original_dir):
    class_dir = os.path.join(original_dir, class_name)
    if not os.path.isdir(class_dir):
        continue
    images = [img for img in os.listdir(class_dir) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(images)
    split_idx = int(len(images) * (1 - split_ratio))
    train_imgs = images[:split_idx]
    val_imgs = images[split_idx:]

    os.makedirs(os.path.join(split_dir, 'train', class_name), exist_ok=True)
    os.makedirs(os.path.join(split_dir, 'val', class_name), exist_ok=True)

    for img in train_imgs:
        shutil.copy(os.path.join(class_dir, img), os.path.join(split_dir, 'train', class_name, img))
    for img in val_imgs:
        shutil.copy(os.path.join(class_dir, img), os.path.join(split_dir, 'val', class_name, img))

TOMATO_CLASSES = [
    'Tomato_Bacterial_spot',
    'Tomato_Early_blight',
    'Tomato_Late_blight',
    'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot',
    'Tomato_Spider_mites_Two_spotted_spider_mite',
    'Tomato__Target_Spot',
    'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus',
    'Tomato_healthy',
]

def random_orient_augment(img_path, img_size):
    img = load_img(img_path, target_size=img_size)
    img_arr = img_to_array(img)
    if random.random() > 0.5:
        img_arr = tf.image.flip_left_right(img_arr)
    if random.random() > 0.5:
        img_arr = tf.image.flip_up_down(img_arr)
    k = random.randint(0, 3)
    img_arr = tf.image.rot90(img_arr, k=k)
    return array_to_img(img_arr)

for split in ['train', 'val']:
    src_dir = f'/content/plantvillage_split/{split}'
    dst_tomato = f'/content/data/{split}/Tomato'
    dst_non_tomato = f'/content/data/{split}/Non-Tomato'
    os.makedirs(dst_tomato, exist_ok=True)
    os.makedirs(dst_non_tomato, exist_ok=True)
    for class_name in os.listdir(src_dir):
        class_path = os.path.join(src_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        for img in os.listdir(class_path):
            if not img.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue
            src_img = os.path.join(class_path, img)
            try:
                aug_img = random_orient_augment(src_img, IMG_SIZE)
                if class_name in TOMATO_CLASSES:
                    dst = dst_tomato
                    new_img_name = f"{class_name}_{img}"
                else:
                    dst = dst_non_tomato
                    new_img_name = f"{class_name}_{img}"
                dst_img = os.path.join(dst, new_img_name)
                aug_img.save(dst_img)
            except Exception as e:
                continue
print('Train and Test Split is Done!')

Train and Test Split is Done!


In [8]:
num_classes = 2
def build_binary_leaf_classifier(img_size=IMG_SIZE):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=img_size + (3,)),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_binary_leaf_classifier()
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,313,026 (12.64 MB)

 Trainable params: 3,313,026 (12.64 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
BATCH_SIZE = 32

train_dir = '/content/data/train'
val_dir = '/content/data/val'

train_ds = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True
)
val_ds = image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False
)

Found 16504 files belonging to 2 classes.
Found 4134 files belonging to 2 classes.


In [10]:
EPOCHS = 20
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

Epoch 1/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 561s 1s/step - accuracy: 0.7619 - loss: 2.2614 - val_accuracy: 0.8212 - val_loss: 0.4304
Epoch 2/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 561s 1s/step - accuracy: 0.8378 - loss: 0.4000 - val_accuracy: 0.8636 - val_loss: 0.3474
Epoch 3/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 549s 1s/step - accuracy: 0.8706 - loss: 0.3302 - val_accuracy: 0.8754 - val_loss: 0.3267
Epoch 4/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 551s 1s/step - accuracy: 0.8908 - loss: 0.2794 - val_accuracy: 0.9076 - val_loss: 0.2461
Epoch 5/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 555s 1s/step - accuracy: 0.9201 - loss: 0.2106 - val_accuracy: 0.9168 - val_loss: 0.2454
Epoch 6/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 561s 1s/step - accuracy: 0.9339 - loss: 0.1713 - val_accuracy: 0.9180 - val_loss: 0.2546
Epoch 7/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 567s 1s/step - accuracy: 0.9455 - loss: 0.1394 - val_accuracy: 0.9178 - val_loss: 0.2726
Epoch 8/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 535s 1s/step - accuracy: 0.9565 - loss: 0.1176 - val_accu

In [11]:
model.save('/content/leaf_classifier.h5')
print('Leaf Classifier saved!')

Leaf Classifier saved!
